In [1]:
import numpy as np
import pandas as pd

# VGG architecture has proved to worked well on small sized images

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from os.path import join as opj
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pylab
plt.rcParams['figure.figsize'] = 10, 10
%matplotlib inline

In [3]:
train = pd.read_json("../data/processed/train.json")
target_train = train['is_iceberg']
test = pd.read_json("../data/processed/test.json")

Here we are removing the last layer of VGG and putting our sigmoid layer for binary predictions.

In [4]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8424 entries, 0 to 8423
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         8424 non-null   object 
 1   band_1     8424 non-null   object 
 2   band_2     8424 non-null   object 
 3   inc_angle  8424 non-null   float64
dtypes: float64(1), object(3)
memory usage: 263.4+ KB


```errors = 'coerce'```
![image](https://user-images.githubusercontent.com/48716219/91058923-08624f00-e664-11ea-9e55-7c5937bb0422.png)


In [34]:
test['inc_angle'] = pd.to_numeric(test['inc_angle'], errors='coerce')
train['inc_angle'] = pd.to_numeric(train['inc_angle'], errors='coerce') # 133개의 NA
train['inc_angle'] = train['inc_angle'].fillna(method='pad')
X_angle=train['inc_angle']
test['inc_angle'] = pd.to_numeric(test['inc_angle'], errors='coerce')
X_test_angle = test['inc_angle']

In [6]:
# Generate the training data
X_band_1 = np.array([np.array(band).astype(np.float32).reshape(75,75) for band in train['band_1']])
X_band_2 = np.array([np.array(band).astype(np.float32).reshape(75,75) for band in train['band_2']])
X_band_3 = (X_band_1 + X_band_2)/2

X_train = np.concatenate([X_band_1[:,:,:, np.newaxis],
                         X_band_2[:,:,:, np.newaxis],
                         X_band_3[:,:,:, np.newaxis]], axis=-1)

In [7]:
# test data
X_band_test_1 = np.array([np.array(band).astype(np.float32).reshape(75,75) for band in test['band_1']])
X_band_test_2 = np.array([np.array(band).astype(np.float32).reshape(75,75) for band in test['band_2']])
X_band_test_3 = (X_band_test_1 + X_band_test_2) / 2
X_test = np.concatenate([X_band_test_1[:,:,:, np.newaxis],
                        X_band_test_2[:,:,:, np.newaxis],
                        X_band_test_3[:,:,:, np.newaxis]], axis=-1)

In [19]:
# import keras
from matplotlib import pyplot as plt
from tensorflow.keras.optimizers import RMSprop, Adam, SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation, LSTM
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.layers import BatchNormalization, Concatenate, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras import initializers
from tensorflow.keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing import image

In [9]:
# Data Aug for multi-input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
batch_size = 64

In [10]:
# Define the image transformation here
gen = ImageDataGenerator(horizontal_flip=True,
                        vertical_flip=True,
                        width_shift_range=0.,
                        height_shift_range=0.,
                        channel_shift_range=0,
                        zoom_range=0.2,
                        rotation_range=10)

In [11]:
# Here is the function that mergess our two generators
# We use the exact same Generator with the same random seed for both the y and angle arrays
def gen_flow_for_two_inputs(X1, X2, y):
    genX1 = gen.flow(X1, y, batch_size=batch_size, seed=55)
    genX2 = gen.flow(X1, X2, batch_size=batch_size, seed=55)
    while True:
        X1i = genX1.next()
        X2i = genX2.next()
        # Assert arrays are equal - this was for peace of mind, but slow down training
        yield [X1i[0], X2i[0]], X1i[1]

In [12]:
# Finally create generator
# 인자로 patience를 주는데 안씀
def get_callbacks(filepath, patience=2) :
    es = EarlyStopping('val_loss', patience=10, mode='min')
    msave = ModelCheckpoint(filepath, save_best_only=True)
    return [es, msave]

In [26]:
def getVggAngleModel():
    input_2 = Input(shape=[1], name="angle")
    angle_layer = Dense(1,)(input_2)
    base_model = VGG16(weights='imagenet', include_top=False,
                      input_shape=X_train.shape[1:], classes=1)
    x = base_model.get_layer('block5_pool').output
    
    x = GlobalMaxPooling2D()(x)
    merge_one = concatenate([x, angle_layer])
    merge_one = Dense(512, activation='relu', name='fc2')(merge_one)
    merge_one = Dropout(0.3)(merge_one)
    merge_one = Dense(512, activation='relu', name='fc3')(merge_one)
    merge_one = Dropout(0.3)(merge_one)
    
    predictions = Dense(1, activation='sigmoid')(merge_one)
    
    model = Model(inputs=[base_model.input, input_2], outputs=predictions)
    
    sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss="binary_crossentropy",
                 optimizer=sgd,
                 metrics=['accuracy'])
    return model

In [27]:
# Using K-fold Cross Valitaion with Data Augmentation
def myAngleCV(X_train, X_angle, X_test):
    K=3
    folds = list(StratifiedKFold(n_splits=K, shuffle=True, random_state=16).split(X_train, target_train))
    y_test_pred_log = 0
    y_train_pred_log = 0
    y_valid_pred_log = 0.0*target_train
    for j , (train_idx, test_idx) in enumerate(folds):
        print('\n=========================Fold=', j)
        X_train_cv = X_train[train_idx]
        y_train_cv = target_train[train_idx]
        X_holdout = X_train[test_idx]
        Y_holdout = target_train[test_idx]
        
        # Angle
        X_angle_cv = X_angle[train_idx]
        X_angle_hold = X_angle[test_idx]
        
        # define file path and get callbacks
        file_path = "%s_aug_model_weights.hdf5"%j
        callbacks = get_callbacks(filepath=file_path, patience=5)
        gen_flow = gen_flow_for_two_inputs(X_train_cv, X_angle_cv, y_train_cv)
        galaxyModel = getVggAngleModel()
        galaxyModel.fit_generator(
                gen_flow,
                steps_per_epoch=24,
                epochs=100,
                shuffle=True,
                verbose=1,
                validation_data=([X_holdout,X_angle_hold], Y_holdout),
                callbacks=callbacks)
        
        #getting the best model
        galaxyModel.load_weights(filepath=file_path)
        #getting training score
        score = galaxyModel.evaluate([X_train_cv, X_angle_cv], y_train_cv, verbose=0)
        print("train loss:", score[0])
        print("train accuracy:", score[1])
        #getting test score
        score = galaxyModel.evaluate([X_holdout, X_angle_hold], Y_holdout, verbose=0)
        print('test loss:', score[0])
        print("train accuracy:", score[1])
        
        #getting validation score
        pred_valid = galaxyModel.predict([X_holdout, X_angle_hold])
        y_valid_pred_log[test_idx] = pred_valid.reshape(pred_valid.shape[0])
        
        #Getting Test Scores
        temp_test=galaxyModel.predict([X_test, X_test_angle])
        y_test_pred_log+=temp_test.reshape(temp_test.shape[0])

        #Getting Train Scores
        temp_train=galaxyModel.predict([X_train, X_angle])
        y_train_pred_log+=temp_train.reshape(temp_train.shape[0])
    
    y_test_pred_log=y_test_pred_log/K
    y_train_pred_log=y_train_pred_log/K

    print('\n Train Log Loss Validation= ',log_loss(target_train, y_train_pred_log))
    print(' Test Log Loss Validation= ',log_loss(target_train, y_valid_pred_log))
    
    return y_test_pred_log

In [28]:
preds=myAngleCV(X_train, X_angle, X_test)


=========================Fold= 0
Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/100


ValueError: in user code:

    /home/aiffel0042/anaconda3/envs/aiffel/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:571 train_function  *
        outputs = self.distribute_strategy.run(
    /home/aiffel0042/anaconda3/envs/aiffel/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/aiffel0042/anaconda3/envs/aiffel/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/aiffel0042/anaconda3/envs/aiffel/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/aiffel0042/anaconda3/envs/aiffel/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:531 train_step  **
        y_pred = self(x, training=True)
    /home/aiffel0

In [32]:


#Data Aug for multi-input
from keras.preprocessing.image import ImageDataGenerator
batch_size=64
# Define the image transformations here
gen = ImageDataGenerator(horizontal_flip = True,
                         vertical_flip = True,
                         width_shift_range = 0.,
                         height_shift_range = 0.,
                         channel_shift_range=0,
                         zoom_range = 0.2,
                         rotation_range = 10)

# Here is the function that merges our two generators
# We use the exact same generator with the same random seed for both the y and angle arrays
def gen_flow_for_two_inputs(X1, X2, y):
    genX1 = gen.flow(X1,y,  batch_size=batch_size,seed=55)
    genX2 = gen.flow(X1,X2, batch_size=batch_size,seed=55)
    while True:
            X1i = genX1.next()
            X2i = genX2.next()
            #Assert arrays are equal - this was for peace of mind, but slows down training
            #np.testing.assert_array_equal(X1i[0],X2i[0])
            yield [X1i[0], X2i[1]], X1i[1]

# Finally create generator
def get_callbacks(filepath, patience=2):
   es = EarlyStopping('val_loss', patience=10, mode="min")
   msave = ModelCheckpoint(filepath, save_best_only=True)
   return [es, msave]


def getVggAngleModel():
    input_2 = Input(shape=[1], name="angle")
    angle_layer = Dense(1, )(input_2)
    base_model = VGG16(weights='imagenet', include_top=False, 
                 input_shape=X_train.shape[1:], classes=1)
    x = base_model.get_layer('block5_pool').output
    

    x = GlobalMaxPooling2D()(x)
    merge_one = concatenate([x, angle_layer])
    merge_one = Dense(512, activation='relu', name='fc2')(merge_one)
    merge_one = Dropout(0.3)(merge_one)
    merge_one = Dense(512, activation='relu', name='fc3')(merge_one)
    merge_one = Dropout(0.3)(merge_one)
    
    predictions = Dense(1, activation='sigmoid')(merge_one)
    
    model = Model(inputs=[base_model.input, input_2], outputs=predictions)
    
    sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='binary_crossentropy',
                  optimizer=sgd,
                  metrics=['accuracy'])
    return model


#Using K-fold Cross Validation with Data Augmentation.
def myAngleCV(X_train, X_angle, X_test):
    K=3
    folds = list(StratifiedKFold(n_splits=K, shuffle=True, random_state=16).split(X_train, target_train))
    y_test_pred_log = 0
    y_train_pred_log=0
    y_valid_pred_log = 0.0*target_train
    for j, (train_idx, test_idx) in enumerate(folds):
        print('\n===================FOLD=',j)
        X_train_cv = X_train[train_idx]
        y_train_cv = target_train[train_idx]
        X_holdout = X_train[test_idx]
        Y_holdout= target_train[test_idx]
        
        #Angle
        X_angle_cv=X_angle[train_idx]
        X_angle_hold=X_angle[test_idx]

        #define file path and get callbacks
        file_path = "%s_aug_model_weights.hdf5"%j
        callbacks = get_callbacks(filepath=file_path, patience=5)
        gen_flow = gen_flow_for_two_inputs(X_train_cv, X_angle_cv, y_train_cv)
        galaxyModel= getVggAngleModel()
        galaxyModel.fit_generator(
                gen_flow,
                steps_per_epoch=24,
                epochs=100,
                shuffle=True,
                verbose=1,
                validation_data=([X_holdout,X_angle_hold], Y_holdout),
                callbacks=callbacks)

        #Getting the Best Model
        galaxyModel.load_weights(filepath=file_path)
        #Getting Training Score
        score = galaxyModel.evaluate([X_train_cv,X_angle_cv], y_train_cv, verbose=0)
        print('Train loss:', score[0])
        print('Train accuracy:', score[1])
        #Getting Test Score
        score = galaxyModel.evaluate([X_holdout,X_angle_hold], Y_holdout, verbose=0)
        print('Test loss:', score[0])
        print('Test accuracy:', score[1])

        #Getting validation Score.
        pred_valid=galaxyModel.predict([X_holdout,X_angle_hold])
        y_valid_pred_log[test_idx] = pred_valid.reshape(pred_valid.shape[0])

        #Getting Test Scores
        temp_test=galaxyModel.predict([X_test, X_test_angle])
        y_test_pred_log+=temp_test.reshape(temp_test.shape[0])

        #Getting Train Scores
        temp_train=galaxyModel.predict([X_train, X_angle])
        y_train_pred_log+=temp_train.reshape(temp_train.shape[0])

    y_test_pred_log=y_test_pred_log/K
    y_train_pred_log=y_train_pred_log/K

    print('\n Train Log Loss Validation= ',log_loss(target_train, y_train_pred_log))
    print(' Test Log Loss Validation= ',log_loss(target_train, y_valid_pred_log))
    return y_test_pred_log

In [35]:
preds=myAngleCV(X_train, X_angle, X_test)


===================FOLD= 0
Epoch 1/100
24/24 [==============================] - 3s 122ms/step - loss: 0.6832 - accuracy: 0.6170 - val_loss: 0.3652 - val_accuracy: 0.8449
Epoch 2/100
24/24 [==============================] - 3s 117ms/step - loss: 0.4112 - accuracy: 0.7956 - val_loss: 0.3121 - val_accuracy: 0.8561
Epoch 3/100
24/24 [==============================] - 3s 118ms/step - loss: 0.3183 - accuracy: 0.8425 - val_loss: 0.2808 - val_accuracy: 0.8654
Epoch 4/100
24/24 [==============================] - 3s 115ms/step - loss: 0.2960 - accuracy: 0.8609 - val_loss: 0.2741 - val_accuracy: 0.8748
Epoch 5/100
24/24 [==============================] - 3s 116ms/step - loss: 0.2883 - accuracy: 0.8698 - val_loss: 0.2510 - val_accuracy: 0.8972
Epoch 6/100
24/24 [==============================] - 3s 106ms/step - loss: 0.2615 - accuracy: 0.8833 - val_loss: 0.2621 - val_accuracy: 0.8879
Epoch 7/100
24/24 [==============================] - 3s 117ms/step - loss: 0.2399 - accuracy: 0.8945 - val_loss: 0

Epoch 9/100
24/24 [==============================] - 3s 109ms/step - loss: 0.2285 - accuracy: 0.8999 - val_loss: 0.2691 - val_accuracy: 0.9045
Epoch 10/100
24/24 [==============================] - 3s 107ms/step - loss: 0.2338 - accuracy: 0.9047 - val_loss: 0.2716 - val_accuracy: 0.8764
Epoch 11/100
24/24 [==============================] - 3s 108ms/step - loss: 0.2076 - accuracy: 0.9124 - val_loss: 0.3028 - val_accuracy: 0.8745
Epoch 12/100
24/24 [==============================] - 3s 109ms/step - loss: 0.1994 - accuracy: 0.9051 - val_loss: 0.2677 - val_accuracy: 0.8951
Epoch 13/100
24/24 [==============================] - 3s 110ms/step - loss: 0.1998 - accuracy: 0.9193 - val_loss: 0.2918 - val_accuracy: 0.8839
Epoch 14/100
24/24 [==============================] - 3s 110ms/step - loss: 0.2004 - accuracy: 0.9163 - val_loss: 0.2685 - val_accuracy: 0.8970
Epoch 15/100
24/24 [==============================] - 3s 115ms/step - loss: 0.1918 - accuracy: 0.9173 - val_loss: 0.2354 - val_accuracy: 

In [37]:
preds

array([0.06502298, 0.3112444 , 0.00160967, ..., 0.01975732, 0.98871475,
       0.02713252], dtype=float32)

In [38]:
import os

In [44]:
os.listdir('../data/processed/')

['test.json', 'sample_submission.csv', 'train.json']

In [45]:
sub = pd.read_csv('../data/processed/sample_submission.csv')
sub

,id,is_iceberg
0,5941774d,0.5
1,4023181e,0.5
2,b20200e4,0.5
3,e7f018bb,0.5
4,4371c8c3,0.5
...,...,...
8419,16ee9b50,0.5
8420,5a599eb7,0.5
8421,df30d6dd,0.5
8422,18af95b1,0.5


In [46]:
#Submission for each day.
submission = pd.DataFrame()
submission['id']=test['id']
submission['is_iceberg']=preds
submission.to_csv('sub.csv', index=False)